# find git repo
[here](https://github.com/despresj/ece_colab)

In [29]:
!nvidia-smi -L

/bin/bash: nvidia-smi: command not found


In [30]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [31]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  proj_dir = "/content/drive/MyDrive/ece884_project/"
else:
  proj_dir = "../"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
df = pd.read_csv(f"{proj_dir}data_clean/clean.csv").to_numpy()

In [33]:
generator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(98, activation="sigmoid")
])

In [34]:
discriminator = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(300, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(1, activation="sigmoid") # not correct output dim
])

In [35]:
gan = keras.models.Sequential([generator, discriminator])

discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [36]:
codings_size = 98
batch_size = 32 # number of data points to return
dataset = tf.data.Dataset.from_tensor_slices(df).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

In [37]:
# Although this runs it still may have bugs
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        print(f"Epoch {epoch + 1} of {n_epochs}")
        for _ in range(1000):
            X_batch = np.random.permutation(df)[:100]
            # phase 1 - training the discriminator
            noise = tf.random.normal(shape=(100,98))
            generated_data = generator(noise)
            X_fake_and_real = tf.concat([generated_data, X_batch], axis=0)

            y1 = tf.constant([[0.]] * noise.shape[0] + [[1.]] * X_batch.shape[0])
            y1 = np.reshape(y1, (200, 1))

            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            # phase 2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)

In [ ]:
train_gan(gan, df, batch_size, codings_size, n_epochs=100)

Epoch 1 of 100
Epoch 2 of 100
Epoch 3 of 100
Epoch 4 of 100
Epoch 5 of 100
Epoch 6 of 100
Epoch 7 of 100
Epoch 8 of 100
Epoch 9 of 100
Epoch 10 of 100
Epoch 11 of 100
Epoch 12 of 100
Epoch 13 of 100
Epoch 14 of 100
Epoch 15 of 100
Epoch 16 of 100
Epoch 17 of 100
Epoch 18 of 100
Epoch 19 of 100
Epoch 20 of 100
Epoch 21 of 100
Epoch 22 of 100
Epoch 23 of 100
Epoch 24 of 100
Epoch 25 of 100
Epoch 26 of 100
Epoch 27 of 100
Epoch 28 of 100
Epoch 29 of 100
Epoch 30 of 100
Epoch 31 of 100
Epoch 32 of 100
Epoch 33 of 100
Epoch 34 of 100
Epoch 35 of 100
Epoch 36 of 100
Epoch 37 of 100
Epoch 38 of 100
Epoch 39 of 100
Epoch 40 of 100
Epoch 41 of 100
Epoch 42 of 100
Epoch 43 of 100
Epoch 44 of 100
Epoch 45 of 100
Epoch 46 of 100
Epoch 47 of 100
Epoch 48 of 100
Epoch 49 of 100
Epoch 50 of 100
Epoch 51 of 100
Epoch 52 of 100
Epoch 53 of 100
Epoch 54 of 100
Epoch 55 of 100
Epoch 56 of 100
Epoch 57 of 100
Epoch 58 of 100
Epoch 59 of 100
Epoch 60 of 100
Epoch 61 of 100
Epoch 62 of 100
Epoch 63 of 100
E

In [ ]:
import os
import re
gans_saved = os.listdir(f"{proj_dir}saved_models/")
model_number = [int(re.sub("[^\d\.]", "", x)) for x in gans_saved]
last_model = max(model_number)

In [ ]:
generator.save(f"{proj_dir}saved_models/gan{last_model+1}")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/ece884_project/saved_models/gan4/assets


In [ ]:
noise = tf.random.normal(shape=[batch_size, codings_size])
generated_data = generator(noise)
generated_data

<tf.Tensor: shape=(32, 98), dtype=float32, numpy=
array([[0.0000000e+00, 0.0000000e+00, 2.1571994e-29, ..., 0.0000000e+00,
        3.5719701e-37, 2.7232075e-33],
       [0.0000000e+00, 0.0000000e+00, 9.2060775e-30, ..., 0.0000000e+00,
        1.1374015e-37, 4.3044027e-34],
       [0.0000000e+00, 0.0000000e+00, 2.0418925e-33, ..., 0.0000000e+00,
        0.0000000e+00, 1.1268122e-37],
       ...,
       [0.0000000e+00, 0.0000000e+00, 1.6029550e-35, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 1.2660697e-35, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 3.9409720e-34, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)>